# Problema del commesso viaggatore (TSP)

## Definizione
### Dato un grafo $G(V,A)$ a cui è associato un costo $d_{ij}$ ad ogni arco $(i,j) \in A$. Il problema del commesso viaggiatore consiste nel cercare il circuito Hamiltoniano di costo minimo.
### - Un ciclo Hamiltoniano è un ciclo che attraversa tutti i nodi del grafo una ed una sola volta.
### - Il costo di un ciclo Hamiltoniano è dato dalla somma dei costi degli archi che lo compongono.

## Definizione

## Variabili di decisione

### <span style="color:purple">$x_{ij} \quad (i,j) \in A $ </span>  - variabile binaria uguale a $1$ se l'arco $(i,j)$ appartiene al circuito hamiltoniano, 0 altrimenti.


### Funzione obiettivo
Minimizza il costo totale del circuito hamiltoniano

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in A} d_{ij} \cdot x_{ij}
\tag{0}
\end{equation}

### Constraints 
- **Vincoli di assegnamento**. In una soluzione ammissibile (circuito hamiltoniano) ogni nodo deve avere esattamente un arco entrante ed esattamente un arco uscente.

\begin{equation}
\sum_{i \in V, \ i \neq j} x_{ij} = 1 \quad \quad j \in V 
\tag{1}
\end{equation}

\begin{equation}
\sum_{i \in V, \ i \neq j} x_{ji} = 1 \quad \quad j \in V 
\tag{2}
\end{equation}

- **Vincoli di assenza di sottogiri**. In una soluzione ammissibile (circuito hamiltoniano) non ci possono essere cicli su un sottoinsime proprio dell'insieme dei nodi $V$.

\begin{equation}
\sum_{i,j \in S, \ (i \neq j)}x_{i,j} \leq |S|-1 \quad \quad  S \subset V
\tag{3}
\end{equation}

## Istanza

### Individuare il circuito hamiltoniano minimo che ci consente di visitare tutti i capoluoghi di regione italiani

### Librerie utilizzate: i) Folium per creare la mappa; ii) Gurobi per risolvere il modello MIP.

In [1]:
import json

capoluoghi_regioni_json = json.load(open('capoluoghi_regioni.json'))

capoluoghi = []
coordinate = {}
for regione in capoluoghi_regioni_json:
    capoluogo = capoluoghi_regioni_json[regione]['capoluogo']
    capoluoghi.append(capoluogo)
    coordinate[capoluogo] = (float(capoluoghi_regioni_json[regione]['lat']), float(capoluoghi_regioni_json[regione]['long']))

### Calcola le distanze per ogni coppia di città

In [2]:
import math
from itertools import combinations

def distance(city1, city2):
    c1 = coordinate[city1]
    c2 = coordinate[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1 in capoluoghi for c2 in capoluoghi if c1 != c2 }

## Importa la libreria GRB inizializza il problema e definisce le variabili

In [3]:
import gurobipy as gp
from gurobipy import GRB

import numpy as np

# Inizializza il modelo
mod = gp.Model('TSP')

# Crea le varibili di decisione
vars = mod.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

Academic license - for non-commercial use only - expires 2021-06-05
Using license file c:\gurobi911\gurobi.lic


## Vincoli di assegnamento

In [4]:
outstar = mod.addConstrs(vars.sum(i, '*') == 1 for i in capoluoghi)

instar = mod.addConstrs(vars.sum('*', i) == 1 for i in capoluoghi)

## Callback Function

### Ogni volta che viene trovata una soluzione intera, la callback function cerca un subtour violato e aggiunge il vincolo di eliminazione corrispondente come "lazy constraint".


In [5]:
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # preleva la soluzione corrente
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i,j) for i, j in model._vars.keys() if vals[i,j] > 0.5)
        # cerca il ciclo di lunghezza minima nella soluzione
        tour = subtour(selected)
        if len(tour) < len(capoluoghi):
            # aggiunge il vincolo di eliminazione di sottogiro
            model.cbLazy(gp.quicksum(model._vars[i,j] for i in tour for j in tour if i != j )
                         <= len(tour)-1)

def subtour(edges):
    unvisited = capoluoghi[:]
    cycle = capoluoghi[:] 
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

## Risolve il modello utilizzando i lazy constraints

In [6]:
mod._vars = vars
mod.Params.lazyConstraints = 1
mod.optimize(subtourelim)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 40 rows, 380 columns and 760 nonzeros
Model fingerprint: 0xa184c3fe
Variable types: 0 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.02s
Presolved: 40 rows, 380 columns, 760 nonzeros
Variable types: 0 continuous, 380 integer (380 binary)

Root relaxation: objective 3.237827e+01, 34 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   34.00998    0   24          -   34.00998      -     -    0s
     0     0   36.77694    0   20          -   36.77694   

## Preleva la soluzione e verifica che sia priva di cicli

In [7]:
vals = mod.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capoluoghi)

In [11]:
tour

['Napoli',
 'Campobasso',
 "L'Aquila",
 'Ancona',
 'Trieste',
 'Venezia',
 'Trento',
 'Milano',
 'Aosta',
 'Torino',
 'Genova',
 'Bologna',
 'Firenze',
 'Perugia',
 'Roma',
 'Cagliari',
 'Palermo',
 'Catanzaro',
 'Bari',
 'Potenza']

In [15]:
import folium
map = folium.Map(location=[43,12], zoom_start = 5)
for city in tour:
    folium.Marker(location = coordinate[city], tooltip = city).add_to(map)

points = []
for city in tour:
  points.append(coordinate[city])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map